In [7]:
import pandas as pd
import os
import random
import numpy as np
import math

In [8]:
def limpar_precos(dataframe,meses_antes_de_setembro=0):
    for i in range(len(dataframe)):
        if ((i - meses_antes_de_setembro) % 3 != 0):
            dataframe = dataframe.drop(i)
    dataframe = dataframe.iloc[::-1]
    dataframe = dataframe.reset_index(drop=True)
    lista = []
    for i in range(len(dataframe)):
        lista.append(int(dataframe.iloc[i]["Último"].replace(",","")))
    dataframe['Preço Lote'] = lista
    dataframe = dataframe.drop(columns=['Último','Abertura','Máxima','Mínima','Vol.','Var%'])
    return dataframe

def importar_dados(nome_arquivo,sheetname=0):
    ponto = nome_arquivo.find('.')
    formato = nome_arquivo[ponto + 1:]
    if (formato == 'xlsx' or formato == 'xls'):
        return pd.read_excel(os.path.join(mypath,nome_arquivo),sheet_name=sheetname)
    if (formato == 'csv'):
        return pd.read_csv(os.path.join(mypath,nome_arquivo))
    else:
        print('formato não definido na função')

def limpar_balanco(dataset,linhas):
    linhas_string = []
    for i in linhas:
        linhas_string.append(str(i))
        dat = pd.DataFrame(columns=linhas_string)
    for i in range(len(linhas)):
        dat[linhas_string[i]] = dataset.iloc[linhas[i] - 2]
    dat = dat.reset_index(drop=True)
    dat.columns = dat.iloc[0]
    dat = dat[1:]
    datas = []
    for i in dataset.iloc[0]:
        datas.append(i)
    datas = datas[1:]
    dat.insert(0,'Datas',datas,True)
    dat = dat.iloc[::-1]
    dat = dat.reset_index(drop=True)
    return dat

def carregar_balanco(filename,linhas,sheet=0):
    linhas2 = []
    linhas1 = []
    planilha2 = False
    for i in linhas:
        if (i > 57):
            linhas2.append(i)
            planilha2 = True 
        else:
            linhas1.append(i)
    df1 = importar_dados(filename,sheet)
    if planilha2:
        linhas22 = []
        for i in linhas2:
            linhas22.append(i - 55)
        df2 = importar_dados(filename,'Dem. Result.')
        if (linhas1 != []):
            return pd.merge(limpar_balanco(df1,linhas1), limpar_balanco(df2,linhas22), how = 'left', on = 'Datas')
        return limpar_balanco(df2,linhas22)
    return limpar_balanco(df1,linhas)

def lista_balancos(nomes,objeto):
    lista = []
    for nome in nomes:
        lista.append(objeto.get_data(nome))
    return lista

def lista_precos(nomes):
    lista = []
    for nome in nomes:
        lista.append(limpar_precos(importar_dados(nome + ' Dados Históricos.csv')))
    return lista


class balancos:
    def __init__(self,nomes,linhas,sheet=1):
        if (type(linhas[0]) is str and sheet == 1):
            numeros = []
            for i in linhas:
                numeros.append(indicadores(i))
            linhas = numeros
        lista_dataframes = []
        if (sheet == 1):
            for i in nomes:
                lista_dataframes.append(carregar_balanco('balanco ' + i + '.xls',linhas))
        if (sheet == 2):
            if (type(linhas[0]) is str):
                numeros = []
                for i in linhas:
                    numeros.append(indicadores(i) - 55)
                linhas = numeros
            for i in nomes:
                lista_dataframes.append(carregar_balanco('balanco ' + i + '.xls',linhas,'Dem. Result.'))
        self.df = lista_dataframes
        self.linhas = linhas
        self.nomes = nomes
    def get_data(self,nome):
        i = self.nomes.index(nome)
        return self.df[i]
    
    

In [9]:
def criar_populacao(genes,individuos):
    populacao = []
    for _ in range(individuos):
        individuo = []
        for _ in range(genes):
            gene = random.randrange(-10,11)
            individuo.append(gene)
        populacao.append(individuo)
    return populacao

def nova_geracao(geracao,aptidoes):
    apt_media = sum(aptidoes) / len(aptidoes)
    #print('aptidao media: ',apt_media)
    #print('maior aptidao: ',max(aptidoes))
    #print('aptidões: ',aptidoes)
    pesos = []
    for valor in aptidoes:
        if valor > 0:
            pesos.append(valor)
        else:
            pesos.append(0)
    #print('pesos: ',pesos)
    populacao = []
    for _ in range(len(geracao)):
        ind = []
        pai = random.choices(geracao,pesos)[0]
        #print('pai: ',pai)
        for gene in pai:
            #ind.append(int(random.triangular(-10,11,gene)))
            ind.append(round(np.random.normal(gene,1,1)[0],0))
        #print('ind',ind)
        populacao.append(ind)
    #print('população: ',populacao)
    return populacao

def calcular_aptidao(todos_balancos,todos_precos,populacao,shares,empresas,debugar = False,aprender = True):
    aptidoes = []
    for individuo in populacao:
        patrimonio = 1000
        if debugar:
            print('individuo: ',individuo)
        if aprender:
            inicio = 15
            fim = len(todos_balancos[0]) - 10
        else:
            inicio = len(todos_balancos[0]) - 10
            fim = len(todos_balancos[0]) - 2
        for i in range(inicio,fim):
            if debugar:
                pass
                print('trimestre: ',todos_balancos[0].iloc[i]['Datas'])
                print('patrimonio: ',patrimonio)
            resultados = []
            empresa = 0
            for balanco in todos_balancos:
                #if debugar:
                    #print('empresa: ',empresas[empresa])
                soma = 0
                for j in range(len(individuo)):
                    if math.isnan(balanco.iloc[i][j + 1]) == False:
                        soma += individuo[j] * balanco.iloc[i][j + 1]
                market_value = shares[empresas[empresa]] * todos_precos[empresa].iloc[i]['Preço Lote'] / 100
                resultados.append(soma / market_value)
                #if debugar:
                    #print('soma: ',soma)
                    #print('market value: ',market_value)
                    #print('resultado: ',soma / market_value)
                empresa += 1
            empresas_escolhidas = dict()
            for valor in resultados:
                if valor > 0:
                    empresas_escolhidas[empresas[resultados.index(valor)]] = valor
            pesos = dict()
            total = 0
            for empresa in empresas_escolhidas:
                total += empresas_escolhidas[empresa]
            for empresa in empresas_escolhidas:
                pesos[empresa] = empresas_escolhidas[empresa] / total
            if debugar:
                #print(resultados)
                #print(empresas_escolhidas)
                print(pesos)
                pass
            for empresa in pesos:
                empresa_escolhida = empresas.index(empresa)
                preco_atual = todos_precos[empresa_escolhida].iloc[i]['Preço Lote']
                preco_futuro = todos_precos[empresa_escolhida].iloc[i + 1]['Preço Lote']
                variacao = (preco_futuro - preco_atual) / preco_atual
                lucro = variacao * patrimonio * pesos[empresa]
                patrimonio += lucro
            if debugar:
                #print('empresa escolhida: ',empresa)
                #print('preco atual: ',preco_atual)
                #print('preco futuro: ',preco_futuro)
                #print('variacao: ',variacao)
                #print('lucro: ',lucro)
                #print('patrimonio: ',patrimonio)
                pass
        aptidoes.append(int(patrimonio - 1000))
    if debugar:
        print('trimestre: ',todos_balancos[0].iloc[fim]['Datas'])
        print('patrimonio: ',patrimonio)
    return aptidoes

def calcular_dumb(todos_precos,shares,emp,empresas,debugar = True):
    print('empresa: ',emp)
    patrimonio = 1000
    inicio = len(todos_precos[0]) - 9
    fim = len(todos_precos[0]) - 1
    for i in range(inicio,fim):
        if debugar:
            pass
            print('trimestre: ',todos_balancos[0].iloc[i]['Datas'])
        pesos = {emp:1}
        for empresa in pesos:
            empresa_escolhida = empresas.index(emp)
            preco_atual = todos_precos[empresa_escolhida].iloc[i]['Preço Lote']
            preco_futuro = todos_precos[empresa_escolhida].iloc[i + 1]['Preço Lote']
            variacao = (preco_futuro - preco_atual) / preco_atual
            lucro = variacao * patrimonio * pesos[empresa]
            print('patrimonio: ',patrimonio)
            patrimonio += lucro
            if debugar:
                #print('empresa escolhida: ',empresa)
                #print('preco atual: ',preco_atual)
                #print('preco futuro: ',preco_futuro)
                #print('variacao: ',variacao)
                #print('lucro: ',lucro)
                pass
    print('trimestre: ',todos_balancos[0].iloc[fim]['Datas'])
    print('patrimonio: ',patrimonio)
    
def recomendar(todos_balancos,todos_precos,individuo,shares,empresas):
    resultados = []
    empresa = 0
    for balanco in todos_balancos:
        soma = 0
        for j in range(len(individuo)):
            if math.isnan(balanco.iloc[-1][j + 1]) == False:
                soma += individuo[j] * balanco.iloc[-1][j + 1]
        market_value = shares[empresas[empresa]] * todos_precos[empresa].iloc[-1]['Preço Lote'] / 100
        resultados.append(soma / market_value)
        empresa += 1
    empresas_escolhidas = dict()
    for valor in resultados:
        if valor > 0:
            empresas_escolhidas[empresas[resultados.index(valor)]] = valor
    pesos = dict()
    total = 0
    for empresa in empresas_escolhidas:
        total += empresas_escolhidas[empresa]
    for empresa in empresas_escolhidas:
        pesos[empresa] = empresas_escolhidas[empresa] / total
    return pesos

In [10]:
mypath = r'balancos'
empresas_a_carregar = ['PETR4','ITSA4','VALE3','LREN3','CMIG4','CYRE3']
shares = {'CYRE3':399743000,'ITSA4':8831360000,'PETR4':13044500000,'VALE3':4999040000,'CMIG4':1693360000,'LREN3':988780000}

In [13]:
def treinar(geracoes):
    ind = balancos(empresas_a_carregar,range(3,27),2)
    todos_balancos = lista_balancos(empresas_a_carregar,ind)
    todos_precos = lista_precos(empresas_a_carregar)
    populacao = criar_populacao(24,100)
    aptidoes_medias = []
    maiores_aptidoes = []
    melhores_individuos = []
    for index in range(geracoes):
        print('geracao ',index)
        aptidao_pop = calcular_aptidao(todos_balancos,todos_precos,populacao,shares,empresas_a_carregar,False,True)
        melhor_ind = aptidao_pop.index(max(aptidao_pop))
        melhores_individuos.append(populacao[melhor_ind])
        maiores_aptidoes.append(max(aptidao_pop))
        aptidao_media = sum(aptidao_pop) / len(aptidao_pop)
        aptidoes_medias.append(aptidao_media)
        print('aptidao media: ',aptidao_media)
        #print('maior aptidao: ',max(aptidao_pop))
        populacao = nova_geracao(populacao,aptidao_pop)

    melhor_ind = melhores_individuos[maiores_aptidoes.index(max(maiores_aptidoes))]
    
    return aptidoes_medias,maiores_aptidoes,melhor_ind

In [ ]:
aptidoes_medias,maiores_aptidoes,melhor_ind = treinar(1000)